In [1]:
import os
import sys

In [2]:
!sh del_all.sh

In [3]:
logs_path = 'slurm_logs/'
slurm_path = 'slurm_scripts/'

In [4]:
def qos(i):
    if i < 12:
        return 'deadline', 24
    elif i < 14:
        return 'normal', 16
    elif i < 18:
        return 'm', 12
    elif i < 26:
        return 'm2', 8
    elif i < 42: 
        return 'm3', 4
    elif i < 74:
        return 'm4', 2
    elif i < 138: 
        return 'cpu_qos', 8
    else:
        return 'scavenger', 24


In [5]:

def write_file(name, test_size, qos, time, job_id=0):
    if not os.path.exists(slurm_path):
        os.mkdir(slurm_path)
    with open(os.path.join(slurm_path, '{}_{}_{}.slrm'.format(name, test_size, job_id)), 'w') as file:
        file.write(
'''#!/bin/bash
#SBATCH --job-name={}_{}_{}\n'''.format(name, test_size, job_id))
        if qos == 'deadline':
            file.write('#SBATCH --account=deadline\n')
        file.write('''#SBATCH -c 16
#SBATCH --qos={0}
#SBATCH --partition={1}'''.format(qos, 't4v1,t4v2,rtx6000,a40\n#SBATCH --gres=gpu:1\n' if not qos=='cpu_qos' else 'cpu\n'))
        file.write('''#SBATCH --mem=16G
#SBATCH --time={1}:00:00
#SBATCH --output=slurm_logs/{0}_{2}_{3}.out
#SBATCH --error=slurm_logs/{0}_{2}_{3}.err
#SBATCH --open-mode=append
#SBATCH --signal=B:USR1@120\n'''.format(name, time, test_size, job_id))
        file.write("term_handler() {\n")
        #file.write("{\n")
        file.write('''\tscontrol requeue "${SLURM_JOB_ID}"\n''')
        file.write("}\ntrap term_handler SIGUSR1\n")
        file.write(
'''source ~/.bashrc
conda activate dis
which python

/h/opent03/.conda/envs/dis/bin/python -u run_div.py --name={} --dataset=uci --loader_args='\{"n_components":5\}' --test_size={} --slurm_job_id=$SLURM_JOB_ID &
child="$!"
wait "$child"

            '''.format(name, test_size))


In [6]:
job_counter = 0
for test_size in [50, 100, 200]:
    for name in ['h', 'kl', 'js']:
        for i in range(10):
            q, t = qos(job_counter)
            job_counter += 1
            write_file(name, test_size, q, t, i)

KeyError: '"n_components"'